## Smooth trap data and store resulting dataframe

This code reads in the dataframes with the trap and neural network counts, converts them to a weekly estimate, and smoothes trap data using a 3 week moving average. The resulting dataframes are stored in `data/Smoothed/`

### Start by importing necessary libraries

In [4]:
import os, sys, importlib, glob, json
import pandas as pd
import numpy as np
import itertools, sys
import matplotlib.pyplot as plt

sys.path.append( os.path.abspath(os.path.join('..')) )
import utils.utils as gen_utils
import utils.forecast as forecast

import warnings
warnings.filterwarnings('ignore')

# Opening config file
f = open('../fpaths_config.json')
paths = json.load(f)

raw_data_path = paths['raw_data']
smoothed_data_path = paths['smoothed_data']

## Save weekly raw and simple smoothing using 3wk moving average

Reads in csv files named `[site]_daily_predictions.csv` with columns `[Location, Datetime, Ref, Ref_sd, Neural Network]` and converts daily predictions to weekly predictions. Saves files with raw AGO and with smoothed AGO in `../data/Raw` and `../data/Smoothed`

In [6]:
#Raw AGO
sites = ['Arboleda', 'Playa', 'La_Margarita', 'Villodas']
weathers = ['site', 'airport']
site_weathers = list(itertools.product(sites, weathers))

for site_weather in site_weathers:
    site, weather = site_weather

    if weather=='airport':
        nn_fil_name = '../data/{}_airport_daily_predictions.csv'.format(site)    
    else:
        nn_fil_name = '../data/{}_site_daily_predictions.csv'.format(site)

    site_nn_data = gen_utils.load_csv(nn_fil_name)
    site_nn_data.Datetime = pd.to_datetime(site_nn_data.Datetime)
    
    weekly = site_nn_data.groupby(pd.Grouper(key='Datetime', freq='W-SAT')).agg({'Ref': np.mean,
                                                                                 'Ref_sd': np.mean,
                                                                                 'Neural Network': np.sum})             
    weekly.loc[weekly.index[0], 'Neural Network'] = 3.5*weekly.loc[weekly.index[0], 'Neural Network']
    weekly.loc[weekly.index[-1], 'Neural Network'] = 8/7*weekly.loc[weekly.index[-1], 'Neural Network']
    
    smoothed_nn = forecast.moving_avg(weekly['Neural Network'], n=3)
    weekly['Neural Network'] = smoothed_nn 

    weekly = weekly.assign(Location=site)
    weekly = weekly[['Location', 'Ref', 'Ref_sd', 'Neural Network']]
    weekly.to_csv('{}/{}_{}_raw_weekly_predictions.csv'.format(raw_data_path,site,weather))
    

In [7]:
#Smoothed AGO
sites = ['Arboleda', 'Playa', 'La_Margarita', 'Villodas']
weathers = ['site', 'airport']
site_weathers = list(itertools.product(sites, weathers))

for site_weather in site_weathers:
    site, weather = site_weather

    if weather=='airport':
        nn_fil_name = '../data/{}_airport_daily_predictions.csv'.format(site)    
    else:
        nn_fil_name = '../data/{}_site_daily_predictions.csv'.format(site)

    site_nn_data = gen_utils.load_csv(nn_fil_name)
    site_nn_data.Datetime = pd.to_datetime(site_nn_data.Datetime)
                
    weekly = site_nn_data.groupby(pd.Grouper(key='Datetime', freq='W-SAT')).agg({'Ref': np.mean,
                                                                                 'Ref_sd': np.mean,
                                                                                 'Neural Network': np.sum})             
    #First week only has 2 days and last week only has 6 days
    weekly.loc[weekly.index[0], 'Neural Network'] = 3.5*weekly.loc[weekly.index[0], 'Neural Network']
    weekly.loc[weekly.index[-1], 'Neural Network'] = 8/7*weekly.loc[weekly.index[-1], 'Neural Network']
        
    smoothed_nn = forecast.moving_avg(weekly['Neural Network'], n=3)
    weekly['Neural Network'] = smoothed_nn 

    weekly.Ref = forecast.moving_avg(weekly.Ref, n=3)
    weekly.Ref_sd = forecast.moving_avg(weekly.Ref_sd, n=3)
    
    weekly = weekly.assign(Location=site)
    weekly = weekly[['Location', 'Ref', 'Ref_sd', 'Neural Network']]
    weekly.to_csv('{}/{}_{}_smoothed_weekly_predictions.csv'.format(smoothed_data_path,site,weather))
    